In [ ]:
import speech_recognition as sr
import subprocess
import os
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from word2number import w2n

r = sr.Recognizer()
r.energy_threshold = 2500
mic = sr.Microphone()

# Funciones para recoger el sonido y transformarlo:
def say(text):
    subprocess.call(['say', "-r 180", text])

# obtain audio from the microphone and transcript
def fun_transcript(t=5, p=6, l="en-US"):
    try:
        with mic as source:
            audio = r.listen(source, timeout=t, phrase_time_limit=p)
            transcript = r.recognize_google(audio, language = l)
            return transcript.lower()
    except:
        pass
    
# activate when you call mandy and return
def activate(phrase='mandy'):
    try:
        transcript = fun_transcript(t=5, p=6)
        if phrase in transcript.lower():
            return [True, transcript.lower()]
        else:
            return activate()
    except:
        return [False, ""]

# personaliza el nombre de la persona con la que habla
def name_set():
    try:
        say("Hola colega, no nos conocemos, ¿cual es tu nombre?")
        n = fun_transcript(t=3, p=3)
        if n == None:
            say("Estoy sorda, ¿puedes repetirlo?")
            n = fun_transcript(t=4, p=3)
            return n
        else:
            return n
    except:
        pass

# Funciones de limpieza de la frase
def tokenizes(s):
    return word_tokenize(s)

def stem_and_lemmatize(s):
    return [WordNetLemmatizer().lemmatize(x, "v") for x in tokenizes(s)]

def remove_stopwords(s):
    return " ".join([w for w in stem_and_lemmatize(s) if not w in stopwords.words("english")])

# Llama a la función correspondiente en función de tus palabras:
def pd_fun(trans):
    

# Lista de funciones de pandas
# Show you the list of possible csv and you choice one
def load_csv():
    print(os.listdir("./datasets"))
    say("Los dataset disponibles son: " + " ".join(os.listdir("./datasets")) + " Elige el que quieras que abra")
    try:
        transcript = fun_transcript(t=6, p=6)
        while transcript == None:
            say("Estoy sorda, ¿puedes repetirlo?")
            transcript = fun_transcript(t=5, p=6)
        df = pd.read_csv("./datasets/{}.csv".format(transcript))
        print(transcript)
        display(df.head())
        return df
    except:
        pass

def header():
    try:
        display(df.head())
    except:
        pass

def tailer():
    try:
        display(df.tail())
    except:
        pass
    
def shape():
    try:
        print(df.shape)
        say("Este dataset tiene {} columnas y {} filas".format(df.shape[1], df.shape[0]))
    except:
        pass
    
def dftypes():
    try:
        print(df.dtypes)
        say("Aquí tienes un listado de todas las columnas con sus tipos de datos".format(df.shape[1], df.shape[0]))
    except:
        pass

def dfcolumntype():
    try:
        column = re.search('columns*(.*)', transcript[1]).group(1).strip().split(" ")[0]
        print(df[column].dtypes)
        say("El tipo de data de la columna {} es {}".format(column, df[column].dtypes))
    except:
        pass

def columnas():
    try:
        display(df.columns)
        say("Las columnas son: {}".format(" ".join(list(df.columns))))
    except:
        pass

def iloca():
    try:
        n = w2n.word_to_num(transcript[1].split(" ")[-1])
        display(df.iloc[n])
    except:
        pass

def loca():
    try:
        column = re.search('columns*(.*)', transcript[1]).group(1).strip().split(" ")[0]
        equal = re.search('equal to(.*)', transcript[1]).group(1).strip().split(" ")[0]
        display(df.loc[column == equal])
    except:
        pass

def isnullo():
    try:
        display(df.isnull().sum()[df.isnull().sum() > 0])
    except:
        pass

def dropnulos():
    try:
        column = re.search('columns*(.*)', transcript[1]).group(1).strip().split(" ")[0]
        return df.dropna(subset=[column], inplace=True)
    except:
        pass

def fillnulos():
    try:
        column = re.search('columns*(.*)', transcript[1]).group(1).strip().split(" ")[0]
        values = ""
        return df[column].fillna(value = values)
    except:
        pass
    
def changetype():
    try:
        column = re.search('columns*(.*)', transcript[1]).group(1).strip().split(" ")[0]
        types = ""
        return df[column].astype(types)
    except:
        pass

def renombrar():
    try:
        old = ""
        new = ""
        return df.rename(columns={old: new})
    except:
        pass

def newindex():
    try:
        column = re.search('columns*(.*)', transcript[1]).group(1).strip().split(" ")[0]
        return df.set_index(column)
    except:
        pass

def descrip():
    # Summary statistics for numerical columns
    try:
        display(df.describe())
    except:
        pass

def media():
    # Returns the mean of all columns
    try:
        display(df.mean())
    except:
        pass
    
def correlacion():
    # Returns the correlation between columns in a DataFrame
    try:
        display(df.corr())
    except:
        pass
    
def counter():
    # Returns the number of non-null values in each DataFrame column
    try:
        display(df.count())
    except:
        pass
    
def maximus():
    # Returns the highest value in each column
    try:
        display(df.max())
    except:
        pass
    
def minimas():
    # Returns the lowest value in each column
    try:
        display(df.min())
    except:
        pass

def mediana():
    # Returns the median of each column
    try:
        display(df.median())
    except:
        pass



In [ ]:
# Código principal
variable = True
name = name_set()
say("Hola {}, que puedo hacer por ti?. Di mi nombre y lo que quieres que haga para que me active. Soy un poco rarita, asi que hablame en ingles aunque yo te responda en castellano".format(name))

while variable:
    try:
        df = pd.read_csv("./datasets/marvel.csv")
        transcript = activate()
        print(r.energy_threshold)
        if transcript[0] == True:
            try:
                print(transcript[1])
                if "read" in transcript[1] and "dataset" in transcript[1]:
                    df = load_csv()
                if "stop" in transcript[1]:
                    variable = False
                clean = remove_stopwords(transcript[1])
                print(clean)
                pd_fun(clean)
            except sr.UnknownValueError:
                say("Vocaliza un poquito, que no hay quien te entienda")
            except sr.RequestError as e:
                print("Could not request results from Mandy service; {0}".format(e))
            
        else:
            pass
    except:
        pass

In [ ]:
remove_stopwords("I can't believe it")

In [252]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.

    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [w for w in nltk.word_tokenize(sent.lower()) if not w in stopwords.words("spanish")]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format

    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.

    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data
            if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return (pad_sequences(xs, maxlen=story_maxlen),
            pad_sequences(xqs, maxlen=query_maxlen), np.array(ys))

RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 3
EPOCHS = 50
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

try:
    path = get_file('mandy.tar.gz', origin='')
except:
    raise

challenge = './mandy/mandy_{}.txt'
with tarfile.open(path) as tar:
    train = get_stories(tar.extractfile(challenge.format('train')))
    test = get_stories(tar.extractfile(challenge.format('test')))
    print(test)

vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = RNN(SENT_HIDDEN_SIZE)(encoded_sentence)

question = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = RNN(QUERY_HIDDEN_SIZE)(encoded_question)

merged = layers.concatenate([encoded_sentence, encoded_question])
preds = layers.Dense(vocab_size, activation='softmax')(merged)

model = Model([sentence, question], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)

print('Evaluation')
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))


RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100
[(['mandy', 'abreme', 'dataset', '.'], ['hacer', 'mandy', '?'], 'load_csv'), (['mandy', 'muestra', 'primeras', 'filas', '.'], ['hacer', 'mandy', '?'], 'header'), (['mandy', 'dime', 'cuales', 'ultimas', 'cinco', 'filas', '.'], ['hacer', 'mandy', '?'], 'tailer'), (['mandy', 'imprime', 'dimension', '.'], ['hacer', 'mandy', '?'], 'shape'), (['mandy', 'muestra', 'dimension', '.'], ['hacer', 'mandy', '?'], 'shape'), (['mandy', 'nombre', 'columnas', '.'], ['hacer', 'mandy', '?'], 'columnas'), (['mandy', 'muestrame', 'tipos', 'datos', 'columna', '.'], ['hacer', 'mandy', '?'], 'dtype'), (['mandy', 'carga', 'dataset', '.'], ['hacer', 'mandy', '?'], 'load_csv'), (['mandy', 'abre', 'csv', '.'], ['hacer', 'mandy', '?'], 'load_csv'), (['mandy', 'muestra', 'ultimas', 'filas', '.'], ['hacer', 'mandy', '?'], 'tailer'), (['mandy', 'dimension', 'dataset', '.'], ['hacer', 'mandy', '?'], 'shape'), (['mandy', 'cuales', 'tipos',

Epoch 50/50
53/53 [==============================] - 0s 6ms/step - loss: 0.0065 - acc: 1.0000 - val_loss: 0.0075 - val_acc: 1.0000
Evaluation
14/14 [==============================] - 0s 2ms/step
Test loss / test accuracy = 0.0184 / 1.0000


In [257]:
def traineural(tran):
    test = [(tokenize(tran), ['hacer', 'mandy', '?'], 'columnas')]
    
    vocab = set()
    for story, q, answer in train + test:
        vocab |= set(story + q + [answer])
    vocab = sorted(vocab)

    # Reserve 0 for masking via pad_sequences
    vocab_size = len(vocab) + 1
    word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
    story_maxlen = max(map(len, (x for x, _, _ in train + test)))
    query_maxlen = max(map(len, (x for _, x, _ in train + test)))

    tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)
    df = pd.DataFrame(index = vocab)
    df["tran"] = model.predict([tx, txq])[1:]
    return df1["tran"].idxmax()
    

In [258]:
print(traineural("Mandy enseñame las primeras filas"))

ValueError: Length of values does not match length of index